In [45]:
!pip install -q --upgrade torch torchvision transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201

In [47]:
from datasets import load_dataset

# Load the emotions datas
dataset = load_dataset("SetFit/emotion")
print(dataset)

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
})


In [48]:
from transformers import AutoTokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [49]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

<ipython-input-51-993bbe7ae7c5>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [52]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.232500,0.207850
2,0.138600,0.160006
3,0.086100,0.160175


TrainOutput(global_step=3000, training_loss=0.21698222478230794, metrics={'train_runtime': 692.2176, 'train_samples_per_second': 69.342, 'train_steps_per_second': 4.334, 'total_flos': 6358888710144000.0, 'train_loss': 0.21698222478230794, 'epoch': 3.0})

In [53]:
trainer.evaluate()

{'eval_loss': 0.16017544269561768,
 'eval_runtime': 10.7648,
 'eval_samples_per_second': 185.79,
 'eval_steps_per_second': 2.973,
 'epoch': 3.0}

In [58]:
from sklearn.metrics import classification_report
import numpy as np

predictions = trainer.predict(tokenized_datasets['test'])
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = tokenized_datasets['test']['label']

print(classification_report(true_labels, predicted_labels))


              precision    recall  f1-score   support

           0       0.97      0.97      0.97       581
           1       0.95      0.95      0.95       695
           2       0.83      0.81      0.82       159
           3       0.94      0.92      0.93       275
           4       0.87      0.92      0.90       224
           5       0.80      0.67      0.73        66

    accuracy                           0.93      2000
   macro avg       0.89      0.87      0.88      2000
weighted avg       0.93      0.93      0.93      2000



In [62]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
encoder.eval()
shots = {
    "joy": [
        "I absolutely loved this!",
        "This makes me so happy."
    ],
    "sadness": [
        "I'm feeling really down today.",
        "This is so depressing."
    ],
    "anger": [
        "This makes me furious!",
        "I can't stand this."
    ],
    "fear": [
        "I'm terrified of what's next.",
        "This scares me so much."
    ],
    "surprise": [
        "Wow, I did not see that coming!",
        "That's a shocking turn of events."
    ],
    "love": [
        "I adore you.",
        "My heart is full of love."
    ],
}


In [63]:
def embed_texts(texts):

    enc = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        out = encoder(**enc).last_hidden_state
    mask = enc.attention_mask.unsqueeze(-1)
    summed = (out * mask).sum(1)
    lengths = mask.sum(1)
    return summed / lengths


prototypes = {}
for label, examples in shots.items():
    emb = embed_texts(examples)
    prototypes[label] = emb.mean(0, keepdim=True)


In [65]:



text = "I can't believe what a wonderful surprise!"
pred_label, similarities = classify(text)
print(f"→ Predicted emotion: {pred_label}")
print(" Similarities:", similarities)


→ Predicted emotion: joy
 Similarities: {'joy': 0.8320035338401794, 'sadness': 0.7193830013275146, 'anger': 0.7771488428115845, 'fear': 0.7037515640258789, 'surprise': 0.819640040397644, 'love': 0.7189898490905762}


In [70]:


test_texts = dataset['test']['text'][:30]
true_labels = dataset['test']['label_text'][:30]

def classify(text):
    q_emb = embed_texts([text])
    sims = {
        label: F.cosine_similarity(q_emb, proto).item()
        for label, proto in prototypes.items()
    }

    return max(sims, key=sims.get), sims

correct = []
incorrect = []

for text, true_label in zip(test_texts, true_labels):
    pred_label, similarities = classify(text)
    entry = {
        "text": text,
        "true_label": true_label,
        "pred_label": pred_label,
        "similarities": similarities
    }
    if pred_label == true_label:
        correct.append(entry)
    else:
        incorrect.append(entry)

print("=== CORRECTLY CLASSIFIED ===")
for e in correct:
    print(f"Text: {e['text']}")
    print(f"True label: {e['true_label']}, Predicted: {e['pred_label']}")
    print(f"Similarities: {e['similarities']}\n")

print("=== MISCLASSIFIED ===")
for e in incorrect:
    print(f"Text: {e['text']}")
    print(f"True label: {e['true_label']}, Predicted: {e['pred_label']}")
    print(f"Similarities: {e['similarities']}\n")


=== CORRECTLY CLASSIFIED ===
Text: im feeling rather rotten so im not very ambitious right now
True label: sadness, Predicted: sadness
Similarities: {'joy': 0.6646848917007446, 'sadness': 0.7591801285743713, 'anger': 0.6592384576797485, 'fear': 0.6923345923423767, 'surprise': 0.7560080885887146, 'love': 0.6483321189880371}

Text: im updating my blog because i feel shitty
True label: sadness, Predicted: sadness
Similarities: {'joy': 0.7109242081642151, 'sadness': 0.789641797542572, 'anger': 0.7315794825553894, 'fear': 0.7154142260551453, 'surprise': 0.7391993403434753, 'love': 0.7086437940597534}

Text: i pay attention it deepens into a feeling of being invaded and helpless
True label: fear, Predicted: fear
Similarities: {'joy': 0.686904788017273, 'sadness': 0.7105132341384888, 'anger': 0.7120696902275085, 'fear': 0.7462707757949829, 'surprise': 0.7423681616783142, 'love': 0.70989990234375}

Text: i feel like reds and purples are just so rich and kind of perfect
True label: joy, Predict